In [1]:
from enum import Enum
import itertools
import copy


with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
"""

Maybe it helps to write some ideas down.
- find arrow + direction. or does direction actually not matter?
- list of coords
- use sets? (at the end)

depending on direction:
- U - (-1,0)
- R - (0,1)
- D - (1,0)
- L - (0,-1)

### Part 1

In [ ]:
class Dirs(Enum):
    U = [-1,0]
    R = [0,1]
    D = [1,0]
    L = [0,-1]

class GuardsGuards():
    def __init__(self,txt):
        self.og_matrix = [[x for x in xs]for xs in txt.splitlines()]
        self.init_guard = (None,None)
        self.obs = []
        for i, row in enumerate(self.txt_matrix):
            for j, c in enumerate(self.txt_matrix[i]):
                match c:
                    case '^':
                        self.init_guard = (i,j)
                        # print(txt_matrix[i][j])
                    case '#':
                        self.obs.append((i,j))
                        # print(txt_matrix[i][j])
        self.init_board()
    
    def init_board(self):
        self.guard = self.init_guard
        # print(self.guard)
        # print(self.obs)
        self.paths = set()
        # self.paths_dir = []

    def get_full_path(self,check_loop=False):
        onScreen = True
        while(onScreen):
            # if dir == Dirs.L: dir = Dirs.U
            # else: dir = DIR
            for dir in Dirs:
                res = self.get_path(dir,check_loop)
                self.paths.update(res[0]) # path
                if res[1]: # gone off screen
                    if check_loop: return res[2] # its a loop!
                    print("bye!")
                    onScreen = False
                    break

    def get_path(self, dir, check_loop=False): # guard pos
        # found = False
        # gone = False
        path = []
        path.append(self.guard)
        loop_count = 0
        while(True):
            dx = self.guard[0]+dir.value[0]
            dy = self.guard[1]+dir.value[1]
            if dx < 0 or dx >= len(self.txt_matrix[0]) or dy <0 or dy >= len(self.txt_matrix): # gone
                return [path,True,False]
            if self.txt_matrix[dx][dy] == '#': # obs
                return [path,False,False]
            else:
                self.guard = (dx,dy)
                path.append(self.guard)
                
            if check_loop: 
                # print("checking loops")
                for pos in self.paths:
                    if dir == pos[1] and (dx,dy) == pos[0]:

                # for i in range(len(self.paths)):
                #     for j, pos in enumerate(self.paths[i]):
                #         if dir == self.paths_dir[i] and (dx,dy) == self.paths[i][j]:
                        print("LOOP WE HAVE A PROBLEM.")
                        print(dir, self.paths_dir[i])
                        print((dx,dy),self.paths[i][j])
                        return [path,True,True]
    
    def check_loop(self):
        counter = 0
        print(len(self.paths))
        for pos in self.paths:
            self.txt_matrix[pos[0]][pos[1]] = '#'
            self.init_board()
            # print(new_matrix)
            # run get_full_paths
            counter += self.get_full_path(True)
            self.txt_matrix[pos[0]][pos[1]] = '.'
            # if get_path with check_loop returns true, plus 1 
        return counter


t_input = GuardsGuards(input)
t_input.get_full_path()
# t_input.check_loop()


bye!


4656

### Part 2: (Which is really just the previous part with some minor changes)
Not quite sure how to proceed from here. Got an idea from [this reply](https://www.reddit.com/r/adventofcode/comments/1h7tovg/comment/m0o8qc3/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button) on reddit

- to enter a loop: position that you've been on + same direction
- check all obstacles?


In [161]:
class Dirs(Enum):
    U = [-1,0]
    R = [0,1]
    D = [1,0]
    L = [0,-1]

class GuardsGuards():
    def __init__(self,txt):
        self.og_matrix = [[x for x in xs]for xs in txt.splitlines()]
        self.txt_matrix = copy.deepcopy(self.og_matrix)
        self.init_guard = (None,None)
        self.init_board()
        self.paths = []
    
    def init_board(self):
        self.obs = []
        for i, row in enumerate(self.txt_matrix):
            for j, c in enumerate(self.txt_matrix[i]):
                match c:
                    case '^':
                        self.init_guard = (i,j)
                        # print(txt_matrix[i][j])
                    case '#':
                        self.obs.append((i,j))
                        # print(txt_matrix[i][j])
        self.guard = self.init_guard

    def get_full_path(self,check_loop=False):
        onScreen = True
        current_paths = []
        while(onScreen):
            for dir in Dirs:
                res = self.get_path(dir,current_paths,check_loop)
                current_paths.append(res[0])
                if res[1]: 
                    if check_loop: return res[2]
                    print("bye!")
                    onScreen = False
                    break

        if not check_loop: self.paths = current_paths
    
    def get_path(self, dir, current_paths, check_loop=False): # guard pos
        # found = False
        # gone = False
        path = []
        while(True):
            dx = self.guard[0]+dir.value[0]
            dy = self.guard[1]+dir.value[1]
            if dx < 0 or dx >= len(self.txt_matrix[0]) or dy <0 or dy >= len(self.txt_matrix): # gone
                return [path,True,False]
            if self.txt_matrix[dx][dy] == '#': # obs
                return [path,False,False]
            else:
                self.guard = (dx,dy)
                path.append((self.guard,dir))

            if check_loop: 
                # print("checking loops")
                # print(current_paths)
                for line in current_paths:
                    for pos in line: #)):
                        # print(pos)
                        if dir == pos[1] and (dx,dy) == pos[0]:
                                # print("LOOP WE HAVE A PROBLEM.")
                                # print(dir, pos[1])
                                # print((dx,dy), pos[0])
                                return [path,True,True]
    
    def get_unique_positions(self):
        s = set()
        for path in self.paths:
            poses = [tup[0] for tup in path]
            # print(poses)
            s.update(poses)
        return s # set(itertools.chain.from_iterable([pos[0] for pos in self.paths]))
    
    def print_path(self):
        m = copy.deepcopy(self.txt_matrix)
        for path in self.paths:
            # print(path) 
            for pos in path:
                m[pos[0][0]][pos[0][1]] =  'X'
        
        [print(''.join(row)) for row in m]        
    
    def check_loop(self):
        counter = 0
        paths = self.get_unique_positions()
        print(paths)
        for pos in paths:
            # print(pos)  # ((1, 6), <Dirs.R: [0, 1]>)
            self.txt_matrix[pos[0]][pos[1]] = '#'
            self.init_board()
            # print(self.txt_matrix)
            # run get_full_paths
            counter += self.get_full_path(True)
            self.txt_matrix[pos[0]][pos[1]] = '.'
            # if get_path with check_loop returns true, plus 1 
        return counter


t_input = GuardsGuards(input)
t_input.get_full_path()
c = t_input.check_loop()
print("loop:", c)
len(t_input.get_unique_positions())

bye!
{(26, 21), (113, 26), (86, 72), (32, 101), (78, 68), (24, 97), (98, 46), (70, 64), (89, 77), (71, 29), (63, 25), (76, 107), (36, 71), (17, 58), (28, 67), (47, 80), (119, 105), (6, 48), (17, 94), (63, 61), (58, 95), (83, 39), (29, 68), (80, 113), (81, 78), (41, 42), (93, 52), (53, 16), (45, 12), (64, 25), (117, 37), (22, 28), (54, 97), (14, 24), (73, 110), (120, 42), (93, 88), (106, 107), (66, 71), (46, 13), (86, 49), (32, 78), (97, 58), (98, 23), (24, 74), (89, 54), (36, 48), (55, 61), (47, 57), (18, 30), (17, 50), (119, 82), (79, 46), (71, 42), (57, 107), (17, 71), (36, 84), (49, 103), (82, 51), (9, 67), (29, 45), (94, 25), (40, 54), (41, 19), (100, 105), (33, 15), (6, 61), (52, 28), (58, 108), (2, 27), (61, 113), (62, 78), (22, 5), (14, 1), (74, 52), (34, 16), (93, 65), (66, 48), (39, 94), (85, 61), (86, 26), (45, 25), (32, 55), (51, 68), (89, 31), (109, 46), (35, 60), (87, 107), (47, 34), (66, 84), (26, 11), (99, 81), (32, 91), (78, 58), (119, 59), (79, 23), (45, 110), (24, 87)

4656

Making the code look nicer. Optimization?

In [159]:
class Dirs(Enum):
    U = [-1,0]
    R = [0,1]
    D = [1,0]
    L = [0,-1]

class GuardsGuards():
    def __init__(self,txt):
        self.og_matrix = [[x for x in xs]for xs in txt.splitlines()]
        self.txt_matrix = [row[:] for row in self.og_matrix]
        self.init_guard = (None,None)
        self.paths = []
        self.obs = set()
        self.init_board()
    
    def init_board(self):
        # self.obs = []
        for i, row in enumerate(self.txt_matrix):
            for j, c in enumerate(self.txt_matrix[i]):
                match c:
                    case '^':
                        self.init_guard = (i,j)
                        # print(txt_matrix[i][j])
                    case '#':
                        self.obs.add((i,j))
                        # print(txt_matrix[i][j])
        self.guard = self.init_guard

    def get_full_path(self,check_loop=False):
        onScreen = True
        current_paths = []
        while(onScreen):
            for dir in Dirs:
                res = self.get_path(dir,current_paths,check_loop)
                current_paths.append(res[0])
                if res[1]: 
                    if check_loop: return res[2]
                    print("bye!")
                    onScreen = False
                    break

        if not check_loop: self.paths = current_paths
    
    def get_path(self, dir, current_paths, check_loop=False): # guard pos
        # found = False
        # gone = False
        path = []
        while(True):
            dx,dy = self.guard[0]+dir.value[0], self.guard[1]+dir.value[1]

            if dx < 0 or dx >= len(self.txt_matrix[0]) or dy <0 or dy >= len(self.txt_matrix): # gone
                return [path,True,False]
            if (dx, dy) in self.obs: # obs
                return [path,False,False]
            
            self.guard = (dx,dy)
            path.append((self.guard,dir))

            if check_loop: 
                if any((dx,dy) == pos[0] and dir == pos[1] for line in current_paths for pos in line):
                    return [path,True,True]
                # print("checking loops")
                # print(current_paths)
                # for line in current_paths:
                #     for pos in line: #)):
                #         # print(pos)
                #         if dir == pos[1] and (dx,dy) == pos[0]:
                #                 print("LOOP WE HAVE A PROBLEM.")
                #                 print(dir, pos[1])
                #                 print((dx,dy), pos[0])
                #                 return [path,True,True]
                    
    def get_unique_positions(self):
        return {pos[0] for path in self.paths for pos in path}
        # s = set()
        # for path in self.paths:
        #     poses = [tup[0] for tup in path]
        #     # print(poses)
        #     s.update(poses)
        # return s # set(itertools.chain.from_iterable([pos[0] for pos in self.paths]))
    
    def print_path(self):
        m = copy.deepcopy(self.txt_matrix)
        for path in self.paths:
            # print(path) 
            for pos in path:
                m[pos[0][0]][pos[0][1]] =  'X'
        
        [print(''.join(row)) for row in m]        
    
    def check_loop(self):
        counter = 0
        unique_positions = self.get_unique_positions()
        print(unique_positions)
        for pos in unique_positions:
            # print(pos)  # ((1, 6), <Dirs.R: [0, 1]>)
            # self.txt_matrix[pos[0]][pos[1]] = '#'
            self.obs.add(pos)
            self.init_board()
            # print(self.txt_matrix)
            # run get_full_paths
            counter += self.get_full_path(True)
            # self.txt_matrix[pos[0]][pos[1]] = '.'
            self.obs.remove(pos)
            # if get_path with check_loop returns true, plus 1 
        return counter


t_input = GuardsGuards(input)
t_input.get_full_path()
c = t_input.check_loop()
print("loop:", c)
# print(len(t_input.paths))
# t_input.get_unique_positions()
len(t_input.get_unique_positions())

bye!
{(26, 21), (113, 26), (86, 72), (32, 101), (78, 68), (24, 97), (98, 46), (70, 64), (89, 77), (71, 29), (63, 25), (76, 107), (36, 71), (17, 58), (28, 67), (47, 80), (119, 105), (6, 48), (17, 94), (63, 61), (58, 95), (83, 39), (29, 68), (80, 113), (81, 78), (41, 42), (93, 52), (53, 16), (45, 12), (64, 25), (117, 37), (22, 28), (54, 97), (14, 24), (73, 110), (120, 42), (93, 88), (106, 107), (66, 71), (46, 13), (86, 49), (32, 78), (97, 58), (98, 23), (24, 74), (89, 54), (36, 48), (55, 61), (47, 57), (18, 30), (17, 50), (119, 82), (79, 46), (71, 42), (57, 107), (17, 71), (36, 84), (49, 103), (82, 51), (9, 67), (29, 45), (94, 25), (40, 54), (41, 19), (100, 105), (33, 15), (6, 61), (52, 28), (58, 108), (2, 27), (61, 113), (62, 78), (22, 5), (14, 1), (74, 52), (34, 16), (93, 65), (66, 48), (39, 94), (85, 61), (86, 26), (45, 25), (32, 55), (51, 68), (89, 31), (109, 46), (35, 60), (87, 107), (47, 34), (66, 84), (26, 11), (99, 81), (32, 91), (78, 58), (119, 59), (79, 23), (45, 110), (24, 87)

4656

Interesting how it performed worse. Never ask ChatGPT for things it does not know.

Unfortunately I'm out of steam today to figure out an improved solution.

### Takeaways
- A brief skim over the subreddit taught me about the [Tortoise and Hare Algorithm (Example: liamasman)](https://github.com/liamasman/advent-of-code-2024/blob/main/src/day_six.py) but it's a bit to attempt it now. Good thing to keep in mind though!
- A lot of mistakes were making a complicated data structure and then stumbling through it trying to make sense of it.
- Take a break and come back to read it later. I'll be less vexed this way. (Today's Connections reference!!)